1. pdata1-features1:feature engineering https://www.kaggle.com/quincyqiang/pdata1-features1
2. pdata1-lgb-train：https://www.kaggle.com/quincyqiang/pdata1-lgb-train
2. pdata1-lgb-inference：current notebook

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm 
from glob import glob
from imp import reload
import copy as cp
import sys
sys.path.append('../input/features-util')
# from utils import util
import util
reload(util)
import joblib


In [ ]:
#%%

import os
import pandas as pd
import numpy as np
import lightgbm as lgb
from tqdm import tqdm 
from glob import glob
import joblib
from multiprocessing import Pool, cpu_count
import warnings
warnings.filterwarnings('ignore')

#%%

input_dir = '../input/optiver-realized-volatility-prediction'
features_name_dir='../input/pdata1-features1'
output_dir = './'
model_dir = '../input/pdata1-lgb-train'




path_lst = glob(os.path.join(input_dir,'book_test.parquet/*'))
stock_lst = [os.path.basename(path).split('=')[-1] for path in path_lst]

#%%

df_all_test =  util.gen_data_multi(stock_lst, data_type='test')


train_df = pd.read_pickle(os.path.join(features_name_dir,'train_stock_df.pkl'))
feature_name=[col for col in train_df.columns.values.tolist() if col not in ['time_id', 'target']]
# df_all_test=df_all_test.reset_index(drop=True)
df_all_test=df_all_test.replace([np.inf, -np.inf], np.nan).fillna(method='ffill')

In [ ]:
features = train_df.columns.difference(['time_id', 'target']).tolist()
# features

In [ ]:
df_all_test[feature_name]

In [ ]:
y_preds = np.zeros(len(df_all_test))
model_lst = glob(os.path.join(model_dir,'*model*.pkl'))
for i, model_path in tqdm(enumerate(model_lst)):
    model = joblib.load(model_path)
    y_preds += model.predict(df_all_test[features])
y_preds /= (len(model_lst))

#%%

df_all_test['row_id'] = df_all_test['stock_id'].astype(str)+'-'+df_all_test['time_id'].astype(str)
df_all_test['target'] = y_preds

#%%

submit = df_all_test[['row_id','target']]

#%%

submit.to_csv('submission.csv',index = False)

#%%

submit.head(5)

In [ ]:
model_lst